In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict, load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import ast
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler


2025-05-02 18:23:22.051194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746210202.074110    3927 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746210202.081467    3927 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/cleared-one-class/cleared_one_class.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4191 entries, 0 to 4190
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4191 non-null   object
 1   class   4191 non-null   object
dtypes: object(2)
memory usage: 65.6+ KB


In [3]:
df.head()

,text,class
0,⭐️ кнопка: ⭐️start⭐️(https://t.me/major/start...,соцсети
1,а продолжение где? правильно. в моем сообществ...,соцсети
2,тем временем моя авторская телега уверенно в т...,соцсети
3,"у меня есть двоюродная сестра, у нее есть сын ...",личная жизнь
4,тем временем моя авторская телега уверенно в т...,соцсети


In [4]:
# Инициализируем энкодер
encoder = LabelEncoder()

labels =encoder.fit_transform(df['class'])
df['labels'] = labels
df.head()

,text,class,labels
0,⭐️ кнопка: ⭐️start⭐️(https://t.me/major/start...,соцсети,3
1,а продолжение где? правильно. в моем сообществ...,соцсети,3
2,тем временем моя авторская телега уверенно в т...,соцсети,3
3,"у меня есть двоюродная сестра, у нее есть сын ...",личная жизнь,0
4,тем временем моя авторская телега уверенно в т...,соцсети,3


In [5]:
class_mapping = {label: idx for idx, label in enumerate(encoder.classes_)}
class_mapping

{'личная жизнь': 0,
 'политика': 1,
 'реклама': 2,
 'соцсети': 3,
 'спорт': 4,
 'юмор': 5}

In [6]:
train_df, val_df = train_test_split(df, stratify=df['labels'], random_state=42)

In [7]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset = DatasetDict({
    'train' : train_dataset,
    'val': val_dataset
})

In [8]:
# Токенизация
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1048 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets.set_format('torch')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'class', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3143
    })
    val: Dataset({
        features: ['text', 'class', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1048
    })
})

In [10]:
# 5. Инициализация модели (остается без изменений)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=6
).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, hamming_loss
import numpy as np


# Создание модели
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", 
    num_labels=len(class_mapping)
    )


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # Для multi-class
    
    # Для multi-label (раскомментировать):
    # predictions = (predictions > 0).astype(int)  # Логиты уже через sigmoid
    
    return {
        'f1 macro': f1_score(labels, predictions, average='macro'),
        'f1 micro': f1_score(labels, predictions, average='micro'),
        'f1 weighted': f1_score(labels, predictions, average='weighted'),
        'accuracy': accuracy_score(labels, predictions),
        'precision': precision_score(labels, predictions, average='macro'),
        'recall': recall_score(labels, predictions, average='macro'),
        'Loss': hamming_loss(labels, predictions)
    }

training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    eval_strategy="steps",
    log_level='info',
    do_train=True,  # Ключевое изменение!
    do_eval=True,
    logging_steps=500,
    save_steps=500,
    eval_steps=500,
    fp16=True,  # Если GPU поддерживает
    report_to=["tensorboard"]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using auto half precision backend


In [12]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: class, __index_level_0__, text. If class, __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3,143
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Training with DataParallel so batch size has been adjusted to: 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1,980
  Number of trainable parameters = 177,858,054
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,F1 macro,F1 micro,F1 weighted,Accuracy,Precision,Recall
500,0.653700,0.242366,0.732450,0.757634,0.755641,0.757634,0.744364,0.724059
1000,0.112400,0.257634,0.707916,0.742366,0.739817,0.742366,0.710257,0.715589
1500,0.102300,0.253817,0.724957,0.746183,0.747875,0.746183,0.721187,0.734840


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: class, __index_level_0__, text. If class, __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1048
  Batch size = 16
Saving model checkpoint to /kaggle/working/results/checkpoint-500
Configuration saved in /kaggle/working/results/checkpoint-500/config.json
Model weights saved in /kaggle/working/results/checkpoint-500/model.safetensors
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: class, __index_level_0__, t

TrainOutput(global_step=1980, training_loss=0.2369513193766276, metrics={'train_runtime': 3567.7038, 'train_samples_per_second': 17.619, 'train_steps_per_second': 0.555, 'total_flos': 1.653975493373952e+16, 'train_loss': 0.2369513193766276, 'epoch': 20.0})

In [13]:
model.save_pretrained("/kaggle/working/my_bert_one_classifier")
tokenizer.save_pretrained("/kaggle/working/my_bert_one_classifier")

Configuration saved in /kaggle/working/my_bert_one_classifier/config.json
Model weights saved in /kaggle/working/my_bert_one_classifier/model.safetensors
tokenizer config file saved in /kaggle/working/my_bert_one_classifier/tokenizer_config.json
Special tokens file saved in /kaggle/working/my_bert_one_classifier/special_tokens_map.json


('/kaggle/working/my_bert_one_classifier/tokenizer_config.json',
 '/kaggle/working/my_bert_one_classifier/special_tokens_map.json',
 '/kaggle/working/my_bert_one_classifier/vocab.txt',
 '/kaggle/working/my_bert_one_classifier/added_tokens.json')

In [14]:
print(trainer.evaluate()) 

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: class, __index_level_0__, text. If class, __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 1048
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 3.5223324298858643, 'eval_f1 macro': 0.7393550675551414, 'eval_f1 micro': 0.7624045801526718, 'eval_f1 weighted': 0.7623028247130316, 'eval_accuracy': 0.7624045801526718, 'eval_precision': 0.7388639613534673, 'eval_recall': 0.7403975919996483, 'eval_Loss': 0.23759541984732824, 'eval_runtime': 20.5196, 'eval_samples_per_second': 51.073, 'eval_steps_per_second': 3.216, 'epoch': 20.0}


In [23]:
new_texts = ["Анкара Месси забил гол в ворота франции", "ай Литвин красава ай чисто на кондциях залетел ай да лев", "Америка расширила список санкция против России", "купите макбук за 140к и получите наушники в подарок"]


In [28]:
def predict(text):
    # Токенизация текста
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)
    
    # Предсказание
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Получение метки класса
    probs = torch.softmax(outputs.logits, dim=1)
    pred_class = torch.argmax(probs).item()
    
    id2label = {
        0: "личная жизнь",
        1: "политика", 
        2: "реклама",
        3: "соцсети",
        4: "спорт",
        5: "юмор"
    }

    for class_id, prob in enumerate(probs.cpu().numpy()[0]):
        print(f"{id2label[class_id]}: {prob:.4f}")  # 4 знака после запятой

In [22]:
class_mapping

{'личная жизнь': 0,
 'политика': 1,
 'реклама': 2,
 'соцсети': 3,
 'спорт': 4,
 'юмор': 5}

In [30]:
predict('Спортивная энергия в каждом глотке — заряжайся и побеждай с новым напитком для чемпионов!')

личная жизнь: 0.0000
политика: 0.0000
реклама: 0.0000
соцсети: 0.0000
спорт: 1.0000
юмор: 0.0000


In [31]:
predict('Достигай рекордов с кроссовками, которые выбирают профи!')

личная жизнь: 0.0000
политика: 0.0000
реклама: 0.0000
соцсети: 0.0342
спорт: 0.9656
юмор: 0.0003


In [32]:
predict('Подпитывай чемпионский дух — белковый батончик для твоих тренировок!')

личная жизнь: 0.0000
политика: 0.0000
реклама: 0.0000
соцсети: 0.0000
спорт: 1.0000
юмор: 0.0000
